In [213]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_aer.primitives import Sampler
from qiskit_aer.backends import AerSimulator
from qiskit.circuit.library import TwoLocal
#from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit import transpile, QuantumCircuit
from qiskit_optimization.translators import to_docplex_mp
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader
from qiskit_optimization.translators import from_docplex_mp
from qiskit.circuit import QuantumCircuit, QuantumRegister, ParameterVector

from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader
from qiskit_optimization.translators import from_docplex_mp
import matplotlib.pyplot as plt
import numpy as np

mod = Model()
mod = ModelReader.read_model('../../lp_G0_lp_c/seq_1040_c.lp')

#set obj
#mod.minimize(0.0)
#print(mod.export_as_lp_string())

#convert linear program to qubo
qp = from_docplex_mp(mod)
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
#quboOp, offset = qubo.to_ising()
Q_info = qubo.objective.quadratic.coefficients.copy()
l_diag = qubo.objective.linear

adj = Q_info + Q_info.T

J_ising = Q_info.copy()
J_ising = J_ising/4.0

for k in range(qp.get_num_vars()):
    Q_info[k,k] = l_diag[k]
    adj[k,k] = l_diag[k]

In [214]:
h_ising = np.zeros(adj.shape[0])
for i in range(0, adj.shape[0]):
    h_ising[i] = 0.25 * np.sum(adj[i,:]) + 0.25 * np.sum(adj[i,i])
    
offset_ising = (0.5 * np.sum([Q_info[i,i] for i in range(0, Q_info.shape[0])]) 
+ 0.125 * np.sum([(np.sum(adj[:, i]) - adj[i,i]) for i in range(0, Q_info.shape[0])]))

In [152]:
offset_ising

207.37499999999997

In [232]:
def adapt_H(J, h, x):
    
    J_tr = J.copy()
    h_tr = h.copy()
    
    for i in range(0, J_tr.shape[0]):
        if x[i] == 1.0:
            h_tr[i]*=(-1)
            
    for idx, _ in np.ndenumerate(J_tr.todense()):
        if idx[0] != idx[1]:
            J_tr[idx[0], idx[1]]*=(-1) ** (x[idx[0]] + x[idx[1]])
            
    return J_tr, h_tr

In [245]:
from itertools import product
x_list = [i for i in product(range(2), repeat=5)]

def objective_ising(x, J, h, offset):
    return x @ J @ x + np.dot(h.T, x) + offset

def objective(x, Q):
    return x @ Q @ x

x_tr = [0, 0, 1, 0, 1]
J_tr, h_tr = adapt_H(J_ising, h_ising, x_tr)

ee_org = {}
for x in x_list:
    ee_org["".join([str(i) for i in x])] = objective_ising(np.array([2*i-1 for i in x]), J_ising, h_ising, offset_ising)
    
ee_tr = {}
for x in x_list:
    ee_tr["".join([str(i) for i in x])] = objective_ising(np.array([2*i-1 for i in x]), J_tr, h_tr, offset_ising)
    
ee_bool = {}
for x in x_list:
    ee_bool["".join([str(i) for i in x])] = objective(np.array([i for i in x]), Q_info)

In [243]:
min(ee_org.items(), key=lambda x: x[1])

('00101', -4.400000000000002)

In [246]:
min(ee_tr.items(), key=lambda x: x[1])

('00000', -4.400000000000002)

In [242]:
min(ee_bool.items(), key=lambda x: x[1])

('00101', -4.4)

In [185]:
Q_info_dict = dict(np.ndenumerate(Q_info.todense()))

In [95]:
np.dot(np.dot(np.array(x_tr).T, Q_info.todense()),np.array(x_tr))  

matrix([[0.]])

In [49]:
Q_ising.todense()[1,:]

matrix([[  0.   , 189.6  ,  -5.   ,  24.475,  24.475,  24.475,  24.475,
          24.475,  24.475,  24.475]])

In [122]:
objective(np.array([-1,-1,-1,-1,1,1,1,1,-1,-1]), Q_ising)

1681.1749999999997

In [186]:
dimod.qubo_to_ising(Q_info_dict)

({0: 9.500000000000002,
  1: 10.850000000000001,
  2: 4.0,
  3: 6.600000000000001,
  4: 6.550000000000001},
 {(0, 1): 2.5500000000000003,
  (0, 2): 2.5500000000000003,
  (0, 3): 2.5500000000000003,
  (0, 4): 2.5500000000000003,
  (1, 2): 2.5500000000000003,
  (1, 3): 2.5500000000000003,
  (1, 4): 2.5500000000000003,
  (3, 4): 2.5500000000000003},
 17.1)

In [187]:
Q_ising.todense()

matrix([[ 9.5 ,  2.55,  2.55,  2.55,  2.55],
        [ 0.  , 10.85,  2.55,  2.55,  2.55],
        [ 0.  ,  0.  ,  4.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  6.6 ,  2.55],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  6.55]])

In [188]:
ee_bool

{'00000': 0.0,
 '00001': -2.2,
 '00010': -2.1,
 '00011': 5.9,
 '00100': -2.2,
 '00101': -4.4,
 '00110': -4.300000000000001,
 '00111': 3.6999999999999993,
 '01000': 1.3,
 '01001': 9.3,
 '01010': 9.400000000000002,
 '01011': 27.600000000000005,
 '01100': 9.3,
 '01101': 17.3,
 '01110': 17.400000000000002,
 '01111': 35.60000000000001,
 '10000': -1.4,
 '10001': 6.6,
 '10010': 6.700000000000001,
 '10011': 24.900000000000006,
 '10100': 6.6,
 '10101': 14.6,
 '10110': 14.700000000000001,
 '10111': 32.900000000000006,
 '11000': 10.100000000000001,
 '11001': 28.300000000000004,
 '11010': 28.400000000000006,
 '11011': 56.80000000000001,
 '11100': 28.300000000000004,
 '11101': 46.50000000000001,
 '11110': 46.60000000000001,
 '11111': 75.00000000000001}

In [212]:
ee_org

{'-1-1-1-1-1': -23.025000000000006,
 '-1-1-1-11': -13.750000000000005,
 '-1-1-11-1': -13.650000000000007,
 '-1-1-111': -1.8250000000000064,
 '-1-11-1-1': -17.575000000000003,
 '-1-11-11': -8.300000000000002,
 '-1-111-1': -8.200000000000003,
 '-1-1111': 3.6249999999999982,
 '-11-1-1-1': -6.425000000000001,
 '-11-1-11': 5.4,
 '-11-11-1': 5.5,
 '-11-111': 19.875,
 '-111-1-1': 1.575000000000002,
 '-111-11': 13.400000000000002,
 '-1111-1': 13.500000000000004,
 '-11111': 27.875000000000007,
 '1-1-1-1-1': -9.125000000000002,
 '1-1-1-11': 2.6999999999999993,
 '1-1-11-1': 2.8,
 '1-1-111': 17.175,
 '1-11-1-1': -1.1249999999999982,
 '1-11-11': 10.700000000000003,
 '1-111-1': 10.800000000000002,
 '1-1111': 25.175000000000004,
 '11-1-1-1': 10.025000000000004,
 '11-1-11': 24.400000000000006,
 '11-11-1': 24.5,
 '11-111': 41.425000000000004,
 '111-1-1': 20.575000000000006,
 '111-11': 34.95000000000001,
 '1111-1': 35.050000000000004,
 '11111': 51.97500000000001}